In [25]:
%pip install transformers, einops, accelerate, langchain, bitsandbytes, sentence_transformers --quiet

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: 'transformers,': Expected end or semicolon (after name and no valid version specifier)
    transformers,
                ^


In [ ]:
%pip pypdf, python-dotenv


In [4]:
!pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
     --- ------------------------------------ 1.0/10.8 MB 6.3 MB/s eta 0:00:02
     ---------- ----------------------------- 2.9/10.8 MB 8.0 MB/s eta 0:00:01
     ----------------- ---------------------- 4.7/10.8 MB 8.4 MB/s eta 0:00:01
     ------------------------ --------------- 6.6/10.8 MB 8.6 MB/s eta 0:00:01
     ----------------------------- ---------- 7.9/10.8 MB 8.2 MB/s eta 0:00:01
     ------------------------------------ --- 9.7/10.8 MB 8.4 MB/s eta 0:00:01
     ---------------------------------------- 10.8/10.8 MB 8.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.7 GB 8.4 MB/s eta 0:05:22
     ---------------------------------------- 0.0/2.7 GB 10.2 MB/s eta 0:04:23
     ---------------------------------------- 0.0/2.7 GB 1

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.4.1+cu118 requires torch==2.4.1+cu118, but you have torch 2.4.0+cu118 which is incompatible.
torchvision 0.19.1+cu118 requires torch==2.4.1+cu118, but you have torch 2.4.0+cu118 which is incompatible.


In [ ]:
%pip install -U "huggingface_hub[cli]" 
%huggingface-cli login

## 01. Load the Large Language Model


In [1]:
model_id="google/flan-t5-large"

In [2]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline.from_model_id(model_id="google/flan-t5-large", task="text2text-generation", model_kwargs={"temperature": 0, "max_length": 200}, device=0)

d:\AI\LLM-APPs-EXPLORE\.conda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\AI\LLM-APPs-EXPLORE\.conda\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
d:\AI\LLM-APPs-EXPLORE\.conda\Lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` 

## 02. Load the Document

In [10]:
# For PDF files
%pip install pypdf

from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("EECC.pdf")
data = loader.load_and_split()

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


## 03. Document transform


When we deal with long pieces of text, it is necessary to use splitters that convert text into chunks.

In [16]:
from langchain.text_splitter import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
data = loader.load_and_split(splitter)

## 04. Embeddings


Embeddings convert our documents into a numerical format which allows us to perform operations such as semantic search by finding the closest matches in the vector space.



In [11]:
%pip install langchain_huggingface

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [18]:
%pip uninstall keras


^C
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install tf-keras

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_huggingface import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name=model_id)

No sentence-transformers model found with name google/flan-t5-large. Creating a new one with mean pooling.


## 05. Storage


After converting our unstructured data into embeddings, we must store them in a local database - “vector store”. There are many options available for vector stores. A vector store manages the storage of embedded data and performs vector searches on behalf of the user.



In [17]:
from langchain.vectorstores import Chroma

persist_directory = '/kaggle/chroma/' # directory

vectorstore = Chroma.from_documents(data, embeddings, persist_directory= persist_directory)

## 06. Retrieval


An important component of this system is the retriever, which is like an interface to retrieve documents based on an unstructured query. Its primary function is to return or retrieve relevant documents when prompted with a query. While vector stores can serve as the foundation for retrievers, there exist alternative forms of retrievers as well. The basic idea is the utilize the stored embeddings from the vector store along with the retriever to obtain desired results.



In [18]:
retriever = vectorstore.as_retriever()


## 07. Memory


LangChain is able to refer to information introduced earlier in the conversation. It provides a lot of utilities for adding memory management to a system.

In [19]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(llm=llm, memory_key="chat_history", output_key='answer', return_messages=True)

C:\Users\Harun\AppData\Local\Temp\ipykernel_29724\3402598109.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(llm=llm, memory_key="chat_history", output_key='answer', return_messages=True)


## 08. Chain


Using an LLM in isolation is suitable for simple applications, but for more complex applications, chaining is necessary for LLM. Chains enable us to combine multiple components to create a single, coherent application.



In [20]:
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, retriever= retriever, memory=memory)

## 09. Deploying


In [ ]:
while True:
    user_input = input("You: ")
    if user_input.lower() == "exit":
        print("Chatbot: Thanks!")
        break
    result = chain({"question": user_input})  # Update the key to 'question'

    # Print the result dictionary to see its structure
    print("Debug: result dictionary:", result)

    # Access the correct key in the result dictionary
    response_key = "answer"  # Update this to the correct key based on the debug output
    response = result.get(response_key, "No response found")

    print("Chatbot:", response)